# Web scraping hero information
- we will get counter and combo win rate from dota stratz
- we will get laning information from dota stratz too

## get hero info from dota stratz
- get it from https://stratz.com/heroes

In [138]:
import logging
import time
import undetected_chromedriver as uc
import base64

# from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium.common.exceptions as Exceptions
from selenium import webdriver
import math
from tqdm.auto import tqdm
import os
import pickle

In [117]:
profile_folder = "~/.config/google-chrome"
 	
# load options
options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=<profile-folder>')
options.add_argument('--profile-directory=Default')
options.page_load_strategy = "normal"
options.headless = False
browser = uc.Chrome(
            version_main=114,
            options=options,
        )


## Obtain all hero urls

In [5]:
HERO_URL = "https://stratz.com/heroes"

In [6]:
# load page 
browser.get(HERO_URL)
time.sleep(2)

In [33]:
heroelexpath = "//a[@id and //ancestor::div[contains(@class, 'hitagi__sc-1q1tiz9-1 gMqHDI')]]"
eles = browser.find_elements(By.XPATH, heroelexpath)

In [40]:
# get all the heros url 
hero_urls = [t.get_attribute('href') for t in eles]
hero_urls[0]

'https://stratz.com/heroes/73'

In [41]:
# now loop each hero url 
cur_url = next(iter(hero_urls))
browser.get(cur_url)

## Obtain laning info

In [107]:
Lane_rate_info_dict = dict()
Url_name_dict = dict()
# dict[heroname] = {["Pos x Pick Rate" : val], "url": val, }

In [108]:
def update_lane_rate_info_for_one_hero(browser, Lane_rate_info_dict, Url_name_dict, cur_url):
    # browser load 
    browser.get(cur_url)
    name_xp = "//span[ contains(@class, 'lbmJJw')]"
    name_ele = browser.find_element(By.XPATH, name_xp)
    hero_name = name_ele.text
    Lane_rate_info_dict[hero_name] = dict()
    Lane_rate_info_dict[hero_name]['url'] = cur_url
    Url_name_dict[cur_url] = hero_name
    # pos 1 2 3 4 5 
    pos_1_lanexp = "//span[contains(text(), 'Safe Lane')]/../../../../div[2]/div[1]/div[1]"
    pos_2_lanexp = "//span[contains(text(), 'Mid Lane')]/../../../../div[2]/div[1]/div[1]"
    pos_3_lanexp = "//span[contains(text(), 'Off Lane')]/../../../../div[2]/div[1]/div[1]"
    pos_4_lanexp = "//span[contains(text(), 'Soft Support')]/../../../../div[2]/div[1]/div[1]"
    pos_5_lanexp = "//span[contains(text(), 'Hard Support')]/../../../../div[2]/div[1]/div[1]"
    lanexps = {"Pos 1 Pick Rate": pos_1_lanexp,
               "Pos 2 Pick Rate": pos_2_lanexp,
               "Pos 3 Pick Rate": pos_3_lanexp,
               "Pos 4 Pick Rate": pos_4_lanexp,
               "Pos 5 Pick Rate": pos_5_lanexp}
    for key, xp in lanexps.items():
        lane_rate_ele = browser.find_element(By.XPATH, xp)
        lane_rate = round(float(lane_rate_ele.text.strip('%')) / 100.0, 3)
        Lane_rate_info_dict[hero_name][key] = lane_rate

In [74]:
update_lane_rate_info_for_one_hero(browser, Lane_rate_info_dict, Url_name_dict, cur_url)

In [75]:
Lane_rate_info_dict

{'Alchemist': {'url': 'https://stratz.com/heroes/73',
  'Pos 1 Pick Rate': 0.289,
  'Pos 2 Pick Rate': 0.283,
  'Pos 3 Pick Rate': 0.22,
  'Pos 4 Pick Rate': 0.117,
  'Pos 5 Pick Rate': 0.09}}

In [70]:
Url_name_dict

{'https://stratz.com/heroes/73': 'Alchemist'}

In [110]:
# wait for the progress
for cur_url in tqdm(hero_urls):
    update_lane_rate_info_for_one_hero(browser, Lane_rate_info_dict,Url_name_dict, cur_url)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [06:29<00:00,  3.14s/it]


## Obtain winrate info

In [105]:
Hero_versus_with_winrate_dict = dict()
# dict[heroname]["pair_hero_name"] = {versus_win_rate, with_win_rate}

In [131]:
def update_win_rate_info_for_one_hero(browser, Url_name_dict, Hero_versus_with_winrate_dict, cur_url):
    winrate_url = os.path.join(cur_url, "matchups")
    hero_name = Url_name_dict[cur_url]
    Hero_versus_with_winrate_dict[hero_name] = dict()
    # browser load 
    browser.get(winrate_url) 
    eachrowxp = "//a[contains(@class, 'hitagi__sc-1hmgwow-0')]"
    
    test= WebDriverWait(browser, 20).until(EC.visibility_of_element_located((By.XPATH, eachrowxp)))
    time.sleep(5)
    roweles = browser.find_elements(By.XPATH, eachrowxp)
    for tele in roweles:
        pair_url = "/".join(tele.get_attribute('href').split('/')[:-1])
        if pair_url in Url_name_dict:
            pair_hero_name = Url_name_dict[pair_url]
        else:
            raise RuntimeError(f"{pair_url} not found in Url_name_dict")
        Hero_versus_with_winrate_dict[hero_name][pair_hero_name] = dict()
        versus_win_rate_xp = "./div[4]/div[1]"
        with_win_rate_xp = "./div[9]/div[1]"
        versus_win_rate_ele = tele.find_element(By.XPATH, versus_win_rate_xp)
        with_win_rate_ele = tele.find_element(By.XPATH, with_win_rate_xp)
        versus_win_rate = round(float(versus_win_rate_ele.text.strip('%')) / 100.0, 3)
        with_win_rate = round(float(with_win_rate_ele.text.strip('%')) / 100.0, 3)
        Hero_versus_with_winrate_dict[hero_name][pair_hero_name]['Versus Win Rate'] = versus_win_rate
        Hero_versus_with_winrate_dict[hero_name][pair_hero_name]['Match Win Rate'] = with_win_rate

In [134]:
# wait for the progress
for ind, cur_url in enumerate(tqdm(hero_urls)):
    heroname = Url_name_dict[cur_url]
    if len(Hero_versus_with_winrate_dict[heroname]) == 123: # fill up incomplete data
        continue 
    update_win_rate_info_for_one_hero(browser, Url_name_dict, Hero_versus_with_winrate_dict, cur_url)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [05:19<00:00,  2.58s/it]


In [137]:
browser.close()

## Save the intermediate information dict

In [135]:
count = 0
for name, valdic in Hero_versus_with_winrate_dict.items():
    if len(valdic) != 123:
        print(name)
        # print(valdic)
        count += 1
print(f"find {count} incomplete data")

find 0 incomplete data


In [ ]:
# we need to store three dict
Hero_versus_with_winrate_dict
Lane_rate_info_dict
Url_name_dict

In [139]:
with open("../records/hero_versus_with_winrate_dict.pkl", 'wb') as f:
    pickle.dump(Hero_versus_with_winrate_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("../records/lane_rate_info_dict.pkl", 'wb') as f:
    pickle.dump(Lane_rate_info_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

with open("../records/url_name_dict.pkl", 'wb') as f:
    pickle.dump(Url_name_dict, f, protocol=pickle.HIGHEST_PROTOCOL)